In [ ]:
%load_ext lab_black

In [4]:
from pyprojroot import here
import os
from skimpy import clean_columns
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
import shap
import joblib
from joblib import load

path_data = here("./data")
os.chdir(path_data)

data = pd.read_csv("data_er_visits.csv").rename(columns={"Unnamed: 0": "Member ID"})
data_no_member_id = data.drop(columns=["Member ID"])
data_predict = data_no_member_id.drop(columns=["Hospital ID", "ER Visit"])
# Load the model
model = joblib.load("model_drivers.joblib")

Get the actual percentage of members with ER visits in the last 30 days for each hospital

Show them their average percentages for all four variables.

Add some noise for the features by Hospital ID

Then create a function to allow for simulated data with the exact percentage that someone shows

In [6]:
data_no_member_id.groupby("Hospital ID").mean().reset_index().round(2)

,Hospital ID,Age 60+,High Cholesterol,Diabetes,Preventative Services,ER Visit
0,1,0.29,0.41,0.25,0.51,0.28
1,2,0.28,0.40,0.24,0.50,0.27
2,3,0.29,0.40,0.24,0.49,0.28


Step: Create a function that generates 

In [7]:
import pandas as pd
import numpy as np


def adjust_binary_percentages(df, **column_percentages):
    """
    Adjusts the percentage of 1's in each specified column of a binary DataFrame.

    Args:
    df (pd.DataFrame): DataFrame with binary values.
    column_percentages (dict): A dictionary where keys are column names and values are the new desired percentages of 1's.

    Returns:
    pd.DataFrame: Modified DataFrame.
    """

    for column, percentage in column_percentages.items():
        if column not in df.columns:
            raise ValueError(f"Column {column} not found in DataFrame")

        # Calculate current percentage of 1's
        current_percentage = df[column].mean()

        # Calculate the desired number of 1's
        target_count = int(df.shape[0] * percentage)

        # Find indices where changes are needed
        ones_indices = df[df[column] == 1].index
        zeros_indices = df[df[column] == 0].index

        if target_count > ones_indices.size:  # Need to add more 1's
            change_count = target_count - ones_indices.size
            indices_to_change = np.random.choice(
                zeros_indices, change_count, replace=False
            )
            df.loc[indices_to_change, column] = 1
        else:  # Need to remove some 1's
            change_count = ones_indices.size - target_count
            indices_to_change = np.random.choice(
                ones_indices, change_count, replace=False
            )
            df.loc[indices_to_change, column] = 0

    return df

In [26]:
data_predict_adjust = clean_columns(data_predict.copy())
data_predict_adjust = adjust_binary_percentages(
    df=data_predict_adjust, age_60 = .1, high_cholesterol = .1, diabetes = .1, preventative_services=.7
)
data_predict_adjust.rename(columns={
    'age_60': 'Age 60+',
    'high_cholesterol': 'High Cholesterol',
    'diabetes': 'Diabetes',
    'preventative_services': 'Preventative Services'
}, inplace=True)

data_predict_adjust

,Age 60+,High Cholesterol,Diabetes,Preventative Services
0,0,0,0,1
1,1,0,0,0
2,0,0,0,1
3,0,1,0,0
4,0,0,0,1
...,...,...,...,...
9995,0,1,1,1
9996,0,0,0,1
9997,1,0,0,0
9998,0,0,0,0


In [27]:
# Make predictions (opxtional, to evaluate model)
predictions = model.predict(data_predict_adjust)
predictions_pd = pd.DataFrame(predictions).rename(columns = {0:'Predictions'})
predictions_pd

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.


,Predictions
0,0
1,1
2,0
3,0
4,0
...,...
9995,0
9996,0
9997,1
9998,0


In [28]:
data_no_member_id.groupby("Hospital ID").mean().reset_index().round(2)

,Hospital ID,Age 60+,High Cholesterol,Diabetes,Preventative Services,ER Visit
0,1,0.29,0.41,0.25,0.51,0.28
1,2,0.28,0.40,0.24,0.50,0.27
2,3,0.29,0.40,0.24,0.49,0.28


In [29]:
data_predictions_hospital_id = pd.concat([data_no_member_id['Hospital ID'], predictions_pd], axis = 1)
data_predictions_hospital_id.groupby("Hospital ID").mean().reset_index().round(2)


,Hospital ID,Predictions
0,1,0.04
1,2,0.04
2,3,0.04
